PETALS TO THE METAL : FLOWER CLASSIFICATION ON TPU

L'objectif initial de cette compétition Kaggle était donc d'entraîner un modèle de machine learning qui reconnaîtrait les différentes catégories de fleurs à partir d'une image. 

L'aglorithme de deep learning fonctionne par entraînement sur le 'training set' pour pouvoir apprendre à la machine avec des exemples. Ainsi, derrière notre algorithme nous avons des réseaux de neurones qui mettent en relation les données d'entrées : ici les images et en sortie ce que représente l'image. Entre les deux étapes on a les 'couches cachées qui sont l'addition des entrées auxquelles on a effecté à chacune d'entre elle un poids. 

La complexité du deep learning réside donc dans le nombre de neurones à mettre dans le réseau. Lorsqu'il y a trop de neurones, il devient vite difficile d'ajuster les poids et la phase d'apprentissage devient trop complexe pour la machine. En outre, pour un grand nombre de problématiques avec peu de données d'entrées tels que la détection de la fraude, on se limite habituellement à trois couches de neurones. 

La detection d'image demande au contraire un grand nombre d'entrées. En effet, les plus petites images de notre base de données sont de taille 192x192 ce qui fait 36864 pixels. C'est pourquoi Google a mis au point son processeur TPU (Tensor Processing Units) afin d'accélerer les calculs de deep learning. 

Nous n'utiliserons pas le TPU ici (je sais plus pq)...



In [40]:
#On importe les packages 

import numpy as np
import os
from sklearn import svm, metrics
from sklearn.model_selection import GridSearchCV
import pandas as pd
import pandas_tfrecords as pdtfr
from PIL import Image
import cv2
import base64
import io
from _datetime import datetime
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical


In [41]:
print("The current working directory is:")
print(os.getcwd())

The current working directory is:
/Users/lauriannemoriceau


La première problématique auquelle nous avons été confrontée est l'importation des données en format 'tfrecords'. Pour cela, il existe une fonction sur pandas 'tfrecords_to_pandas'. On cree donc un dataframe avec toutes les images pour entraîner notre modèle, ainsi qu'un 'data test' pour tester la qualité de prédiction de l'algorithme.


In [42]:
data = pdtfr.tfrecords_to_pandas("Data/Kaggle/train/00-192x192-798.tfrec")
data = data.append(pdtfr.tfrecords_to_pandas("Data/Kaggle/train/01-192x192-798.tfrec"), ignore_index=True)
data = data.append(pdtfr.tfrecords_to_pandas("Data/Kaggle/train/02-192x192-798.tfrec"), ignore_index=True)
data = data.append(pdtfr.tfrecords_to_pandas("Data/Kaggle/train/03-192x192-798.tfrec"), ignore_index=True)
data = data.append(pdtfr.tfrecords_to_pandas("Data/Kaggle/train/04-192x192-798.tfrec"), ignore_index=True)
data = data.append(pdtfr.tfrecords_to_pandas("Data/Kaggle/train/05-192x192-798.tfrec"), ignore_index=True)
data = data.append(pdtfr.tfrecords_to_pandas("Data/Kaggle/train/06-192x192-798.tfrec"), ignore_index=True)
data = data.append(pdtfr.tfrecords_to_pandas("Data/Kaggle/train/07-192x192-798.tfrec"), ignore_index=True)
data = data.append(pdtfr.tfrecords_to_pandas("Data/Kaggle/train/08-192x192-798.tfrec"), ignore_index=True)
data = data.append(pdtfr.tfrecords_to_pandas("Data/Kaggle/train/09-192x192-798.tfrec"), ignore_index=True)
data = data.append(pdtfr.tfrecords_to_pandas("Data/Kaggle/train/10-192x192-798.tfrec"), ignore_index=True)
data = data.append(pdtfr.tfrecords_to_pandas("Data/Kaggle/train/11-192x192-798.tfrec"), ignore_index=True)
data = data.append(pdtfr.tfrecords_to_pandas("Data/Kaggle/train/12-192x192-798.tfrec"), ignore_index=True)
data = data.append(pdtfr.tfrecords_to_pandas("Data/Kaggle/train/13-192x192-798.tfrec"), ignore_index=True)
data = data.append(pdtfr.tfrecords_to_pandas("Data/Kaggle/train/14-192x192-798.tfrec"), ignore_index=True)
data = data.append(pdtfr.tfrecords_to_pandas("Data/Kaggle/train/15-192x192-783.tfrec"), ignore_index=True)
datatest = pdtfr.tfrecords_to_pandas("Data/Kaggle/val/00-192x192-232.tfrec")

Ca sert à quoi ça : ? 

In [43]:
np.set_printoptions(threshold=np.inf) # in-console display options
pd.set_option('display.max_rows', 1000) # in-console display options

On regroupe les 12753 images selon les 104 classes

In [44]:
data.describe()
count = data.groupby('class').count()
print(count)

        id  image
class            
0      272    272
1       26     26
2       20     20
3       21     21
4      703    703
5       87     87
6       18     18
7      105    105
8       87     87
9       84     84
10     136    136
11      43     43
12      92     92
13     263    263
14     227    227
15      21     21
16      55     55
17      50     50
18      90     90
19      26     26
20      19     19
21      96     96
22      48     48
23      19     19
24      85     85
25      83     83
26      21     21
27      34     34
28     119    119
29     109    109
30     105    105
31      24     24
32      23     23
33      20     20
34      18     18
35      36     36
36      57     57
37      26     26
38      19     19
39      73     73
40      64     64
41      96     96
42      63     63
43     110    110
44      18     18
45     172    172
46     125    125
47     261    261
48     422    422
49     563    563
50     201    201
51     105    105
52     115    115
53     460

Afin de pouvoir entraîner les modèles correctement, nous ne gardons que les classes qui contiennent plus de 200 observations, ce qui fait 14 classes. 

In [45]:
def labelReduction (data): 
    count = data.groupby('class').count()
    count.reset_index(level=0, inplace=True)
    classes = []

    for i in range(0, 104):
        if count['id'][i]  > 200:
            var = count['class'][i]
            classes.append(var)

    for i in range(0, len(data['class'])):
        if not data['class'][i] in classes:
            data.drop([i], inplace = True)
        else:
            print('process running')

Pour l'ordinateur, l'image est un ensemble de valeurs/structures discrètres. Les pixels prennent une valeur d'intensité (entre 0 (le noir) et 255 (le blanc), les valeurs au milieu représentent les niveaux de gris. L'image devient donc une matrice de valeurs. Il faut donc convertir tout d'abord convertir les bytes qui sont l'équivalent du format str sous python pour avoir une image PIL. 

In [46]:
def stringToRGB(base64_string): # converts bytes to RGB/jpeg
    image = io.BytesIO(base64_string)
    image.seek(0)
    return Image.open(image) #PIL function

Une fois les images PIL obtenues on redimensionne les images pour avoir moins de pixels. 

In [47]:
def resize(image): # reduce resolution
    basewidth = 24 #largeur de 24 pixels
    size = image.size[0]
    wpercent = (basewidth / int(size))
    hsize = int((float(image.size[1]) * float(wpercent)))
    img = image.resize((basewidth, hsize), Image.ANTIALIAS)
    return img

In [48]:
def imgShow(data): #displays image
    a = data['image'][50]
    a = stringToRGB(a)
    a.show()


On construit un algorithme intermédiaire pour avoir un réseau plus simple et plus facile à entraîner. Pour cela, on extrait des caractéristiques intermédiaires qui résument les images de fleurs. Ces caractéristiques sont stockés sous forme de liste pour les échantillons 'train' et 'test'.  

In [49]:
def featureExtraction(data, datatest): # convert jpeg into flatten array
    global label, features, features_test, label_test
    label = []
    features = []
    features_test = []
    label_test = []

    for i in range(0, len(data)):
        classes = str(data['class'][i])
        label.append(classes)

    for i in range(0, len(data)):
        image = stringToRGB(data['image'][i])
        image = resize(image)
        image = np.asarray(image) #on convertit en tableau
        image = image.flatten('F') #on applatit ss forme de liste ordonnee en colonnes
        features.append(image)

    for i in range(0, len(datatest)):
        classes_test = str(datatest['class'][i])
        label_test.append(classes_test)

    for i in range(0, len(datatest)):
        image_test = stringToRGB(datatest['image'][i])
        image_test = resize(image_test)
        image_test = np.asarray(image_test)
        image_test = image_test.flatten('F')
        features_test.append(image_test)

In [50]:
features = np.array(features)
features_test = np.array(features_test)
label = np.array(label)
label_test = np.array(label_test)


Data pre-processing ??

In [51]:
imgShow(datatest)
labelReduction(data)
data.reset_index(inplace =True)

print(data)
featureExtraction(data, datatest)

features = features / 255.0
features_test = features_test / 255.0

runtime = []

process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process 

process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process 

process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process 

process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process 

process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process 

...

process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process 

process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process 

process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process 

process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process running
process 

TypeError: unsupported operand type(s) for /: 'list' and 'float'

Nous pouvons commencer la modélisation par l'algorithme d'apprentissage supervisé : SVM. L'objectif du SVM va donc être de séparer les 14 classes pour obtenir une généralisation que l'on appliquera sans connaître la classe d'appartenance de chaque fleur. Les droites sont celles qui maximisent l'écart entre chaque classe mais cela suppose que nos classes sont linéairement séparables.  

A PARTIR DE LA PLUS EN NB

La fonction Grisearch permet de trouver les modèles avec les meilleurs hyperparamètres en comparant les performances de chaque combinaisons. L'ajout d'un kernel 'RBF' permet de palier au problème de non-linéarité par la construction d'un hyperplan pour afin de séparer nos classes.  

In [ ]:
# RBF SVM w/ gridsearch optimization
grid = {
	'C': [0.1, 1, 10, 100, 1000],
	'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
	'kernel': ['rbf']}

startTime = datetime.now()

rbf_SVM = svm.SVC(max_iter = 100)
grid_search = GridSearchCV(rbf_SVM, param_grid = grid, refit = True)
rbf_SVM_fit = grid_search.fit(features, label)

runtime.append(datetime.now() - startTime)

rbf_pred = grid_search.predict(features_test)
print(metrics.classification_report(rbf_pred, label_test))

Nous procédons maintenant à l'implémentation de perceptrons multicouches avec le classifieur 'Multilayer perceptron' (MLP) avec la bibliothèque Keras. Le réseau de neurone est défini par une couche et la fonction d'activation 'ReLU'. La dernière couche doit contenir au moins un neurone par type de fleur où chaque neurone contient la probabilité que la fleur corresponde à sa classe.  
 

In [ ]:
# ANN
label = label.astype(int)

keras_MLP = tf.keras.Sequential([
    tf.keras.layers.Dense(20, activation = 'relu'),
    tf.keras.layers.Dense(104)]) # 1 hidden layer w/ 20 neurons

startTime = datetime.now()

# Opti controls
keras_MLP.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])


On peut à présent appliquer le modèle aux images : 

In [ ]:
# Fit
keras_MLP.fit(features, label, epochs=10)

runtime.append(datetime.now() - startTime)
label = label.astype(int)
label_test = label_test.astype(int)


Pour palier au problème de la taille des images à analyser, on utilise le 'réseau convulsif' qui cherche à analyser les caractéristiques des images. Chaque couche du réseau analyse le contenu de l'image tels que la luminosités des pixels ou encore la corrélation entre chaque pixel pour identifier des structures. Chaque neurone identifie une partie restreinte de l'image auquel il va associer à chaque pixel un poids (par rapport à la luminosité par exemple). Ces poids forment une matrice de convolusion qui va agir sous forme de filtre. Le but étant de réduire le nombre de paramètres du réseau de neurone. On procède également à un sous-échantillonage afin de réduire les nombres de paramètres. 


In [ ]:
# CNN

# preprocessing
features_CNN = np.zeros(shape = (2791, 24, 24, 3))
features_test_CNN = np.zeros(shape = (56, 24, 24, 3))

label = []
label_test = []

for i in range(0, len(data)):
    image = stringToRGB(data['image'][i])
    image = image.convert('RGB')
    image = resize(image)
    image = np.asarray(image)
    features_CNN[i] = image
    classes = str(data['class'][i])
    label.append(classes)

for i in range(0, len(datatest)):
    image_test = stringToRGB(datatest['image'][i])
    image_test = image_test.convert('RGB')
    image_test = resize(image_test)
    image_test = np.asarray(image_test)
    features_test_CNN[i] = image_test
    classes = str(datatest['class'][i])
    label_test.append(classes)

label = to_categorical(label)
label_test = to_categorical(label_test)

img_rows = 24
img_cols = 24

if K.image_data_format() == 'channels_first':
    features_CNN = features_CNN.reshape(features_CNN.shape[0], 3, img_rows, img_cols)
    features_test_CNN = features_test_CNN.reshape(features_test_CNN.shape[0], 3, img_rows, img_cols)
    print(features_CNN.shape)
    input_shape = (3, img_rows, img_cols)
else:
    features_CNN = features_CNN.reshape(features_CNN.shape[0], img_rows, img_cols, 3)
    features_test_CNN = features_test_CNN.reshape(features_test_CNN.shape[0], img_rows, img_cols, 3)
    input_shape = (img_rows, img_cols, 3)


batch_size = 100 #nb points pr calculer les poids : sous-échantillonage
epochs = 10 #nb d'itérations

CNN = tf.keras.Sequential()
CNN.add(Conv2D(32, kernel_size=(3, 3), #3 pixels par 3 pixels sur lequel on fait passer les filtres
                 activation='relu',
                 input_shape=(24, 24, 3))) # convolutional layer #4 couches
CNN.add(Flatten())
CNN.add(Dense(50, activation='relu')) # standard layer
CNN.add(Dense(104, activation='softmax')) # classification layer

CNN.compile(loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

On stock le fit dans history pr pv l'afficher après, validation_data 
CNN evaluate pour avoir les résultats 

In [ ]:
#fit
startTime = datetime.now()

history = CNN.fit(features_CNN, label,
          batch_size=batch_size,
          epochs=epochs,
          verbose=2,
          validation_data=(features_test_CNN, label_test))
score = CNN.evaluate(features_test_CNN, label_test, verbose=0)

runtime.append(datetime.now() - startTime)
